# WuTong V6 Kaggle 一键训练

V6 核心改进：
- 外部数据受控混入（不再淹没原始分布）
- +13 个高级特征（信息熵、编码深度、关键词密度等）
- SMOTE-ENN（合成 + 清洗边界噪声）
- 精调模型参数 + 双集成对比

直接 **Run All** 即可。


In [ ]:
# 1) 克隆/更新代码
from pathlib import Path
import os

repo_dir = Path('/kaggle/working/WuTong')
if repo_dir.exists():
    print('Repo exists, pulling latest...')
    os.system(f'cd {repo_dir} && git pull')
else:
    print('Cloning repo...')
    os.system('git clone https://github.com/alltobebetter/WuTong.git /kaggle/working/WuTong')

%cd /kaggle/working/WuTong
!git log --oneline -n 3


In [ ]:
# 2) 安装依赖
!pip -q install -r requirements.txt
!pip -q install imbalanced-learn  # SMOTE-ENN

import torch
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))


In [ ]:
# 3) 准备数据（优先使用仓库自带数据，其次 /kaggle/input）
from pathlib import Path
import shutil

raw_dir = Path('data/raw')
raw_dir.mkdir(parents=True, exist_ok=True)

# 优先从仓库自带的 data_raw/ 拷贝
repo_data = list(Path('data_raw').glob('*.xlsx')) + list(Path('data_raw').glob('*.csv'))
if repo_data:
    print(f'从仓库 data_raw/ 找到 {len(repo_data)} 个数据文件')
    for src in repo_data:
        dst = raw_dir / src.name
        if not dst.exists():
            shutil.copy2(src, dst)
            print(f'  复制: {src.name}')
else:
    # 回退到 /kaggle/input
    candidates = list(Path('/kaggle/input').rglob('*.xlsx')) + list(Path('/kaggle/input').rglob('*.csv'))
    print(f'从 /kaggle/input 找到 {len(candidates)} 个候选文件')
    for src in candidates[:20]:
        dst = raw_dir / src.name
        if not dst.exists():
            shutil.copy2(src, dst)

print('\ndata/raw files:')
for p in sorted(raw_dir.glob('*')):
    print(f'  - {p.name} ({p.stat().st_size / 1024:.0f} KB)')

if not list(raw_dir.glob('*')):
    raise RuntimeError('未找到任何数据文件！请检查 data_raw/ 或挂载 Kaggle 数据集')


In [ ]:
# 4) 预处理 + 增强 + 外部数据下载
import glob, os

xlsx_files = glob.glob('data/raw/*.xlsx')
csv_files = glob.glob('data/raw/*.csv')
source_files = xlsx_files + csv_files
if not source_files:
    raise RuntimeError('data/raw 没有可用数据')

src = source_files[0]
print('Using source file:', src)
ret = os.system(f'python -u scripts/ingest.py "{src}"')
if ret != 0: raise RuntimeError('ingest 失败')

ret = os.system('python -u scripts/augment_data.py --target-size 30000 --ratio 2.5')
if ret != 0: raise RuntimeError('augment 失败')

# 下载外部数据集（V6 会受控混入，不会淹没原始数据）
ret = os.system('python -u scripts/integrate_csic2010.py')
if ret != 0:
    print('⚠️ 外部数据集下载失败，将仅使用原始+增强数据训练')

print('preprocess pipeline done')


In [ ]:
# 5) V6 训练
import subprocess, sys

print('='*80)
print('🚀 V6 训练开始')
print('='*80)
print('\n核心改进:')
print('  - 外部数据受控混入（每类最多补充 30%）')
print('  - +13 个高级特征（信息熵、编码深度等）')
print('  - SMOTE-ENN（合成 + 清洗边界噪声）')
print('  - 精调参数 + 双集成对比\n')

cmd = [
    sys.executable, '-u', 'scripts/train_v6.py',
    '--version', 'v6.0.0-kaggle',
    '--cv-splits', '10',
]

proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in proc.stdout:
    print(line, end='')

ret = proc.wait()
if ret != 0:
    raise RuntimeError(f'训练失败，退出码: {ret}')

print('\n' + '='*80)
print('✅ V6 训练完成！')
print('='*80)


In [ ]:
# 6) 查看训练结果
from pathlib import Path
import json

manifest_path = Path('models/v6.0.0-kaggle/manifest.json')

if manifest_path.exists():
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest = json.load(f)

    print('='*80)
    print('📊 V6 训练结果')
    print('='*80)
    print(f"\n版本: {manifest['version']}")
    print(f"训练时间: {manifest['trained_at']}")
    print(f"数据量: {manifest['data_rows']} 条")
    print(f"外部数据: {'受控混入 ' + str(manifest.get('external_data_rows', 0)) + ' 条' if manifest.get('external_data') else '否'}")
    print(f"特征数: {len(manifest['feature_list'])} 个")
    print(f"类别数: {len(manifest['classes'])} 类")

    print('\n' + '='*80)
    print('🎯 模型性能')
    print('='*80)

    metrics = manifest['metrics']
    for name in ['xgboost', 'catboost', 'lightgbm']:
        m = metrics[name]
        cv_str = f", CV: {m['cv_accuracy']:.4f}" if m.get('cv_accuracy') else ''
        print(f"  {name}: Acc={m['test_accuracy']:.4f}, F1={m['test_f1']:.4f}{cv_str}")

    e = metrics['ensemble']
    print(f"\n🏆 集成模型（{e['ensemble_type']}）")
    print(f"  准确率: {e['test_accuracy']:.4f} ({e['test_accuracy']*100:.2f}%)")
    print(f"  F1 分数: {e['test_f1']:.4f}")

    if e['test_accuracy'] >= 0.998:
        print('\n  🎉🎉🎉 达到 99.8% 目标！')
    elif e['test_accuracy'] >= 0.995:
        print('\n  🎉🎉 达到 99.5% 目标！')
    elif e['test_accuracy'] >= 0.99:
        print('\n  🎉 达到 99% 目标！')

    print('\n' + '='*80)
else:
    print('❌ 未找到训练结果文件！')


In [ ]:
# 7) 打包下载
from pathlib import Path
import shutil

model_dir = Path('models/v6.0.0-kaggle')
if model_dir.exists():
    print('📦 打包模型文件...')
    archive = shutil.make_archive(
        '/kaggle/working/models_v6.0.0-kaggle', 'zip',
        'models', 'v6.0.0-kaggle'
    )
    print(f'✅ 打包完成: {archive}')
    print(f'\n文件大小: {Path(archive).stat().st_size / 1024 / 1024:.2f} MB')
    print('\n💡 在 Kaggle 右侧 Output 面板下载')
else:
    print('❌ 模型目录不存在！')


## V6 vs V5 改进说明

| 维度 | V5 | V6 |
|------|-----|-----|
| 外部数据 | 全量混入（61k 淹没 29k） | 受控混入（每类最多 30%，过滤不可靠标签） |
| 特征数 | 25 | 38（+13 高级特征） |
| 过采样 | SMOTE（全量拉平） | SMOTE-ENN（合成+清洗，目标 80%） |
| XGBoost 深度 | 12 | 10（防过拟合） |
| 集成策略 | 固定 Stacking | Stacking vs Voting 对比取优 |
| 预期准确率 | 77.46%（实际） | 99%+（修复数据问题后） |
